<h1 align="center">
  <a href="https://uptrain.ai">
    <img width="300" src="https://user-images.githubusercontent.com/108270398/214240695-4f958b76-c993-4ddd-8de6-8668f4d0da84.png" alt="uptrain">
  </a>
</h1>

<h1 style="text-align: center;">Unit-testing of a Fresh LLM model</h1>

**Objective**: Given a golden testing dataset and a new LLM model (or LLM APIs with new prompts), evaluate the performance of the model on the testing dataset and collect cases where the model doesn't perform well.

#### Install required packages

In [1]:
# !pip install uptrain datasets py7zr torch

In [2]:
from datasets import load_dataset
import json
import numpy as np
import os
import uptrain
from rouge import Rouge 
import random
from matplotlib import pyplot as plt
import pandas as pd
import subprocess
import zipfile
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
import torch
import torch.nn.functional as F

PyTorch is available but CUDA is not. Defaulting to SciPy for SVD


#### Following functions are for testing purposes only

In [4]:
samsum_dataset = load_dataset("samsum")
golden_dataset = samsum_dataset["test"][0:20]

  0%|          | 0/3 [00:00<?, ?it/s]

### Define the model to be tested

In [6]:
summary_model = "yasminesarraj/flan-t5-small-samsum"

#### Define a model to genetate sentence embeddings

In [ ]:
sentence_emb_model = "sentence-transformers/paraphrase-MiniLM-L6-v2"

In [9]:
cfg = {
    "checks": [{
        'type': uptrain.Monitor.LLM_EVALUATION,
        "measurable_args": {
            "type": uptrain.MeasurableType.INPUT_FEATURE,
            "feature_name": "dialogues"
        },
        "embedding_model": sentence_emb_model,
        "llm_model_args": {"model_name": summary_model},
        "distance_types": ["cosine_distance", "l2_distance"],
    }],
    "logging_args": {
        "st_logging": True,
        "use_new_handler": True,
        "run_background_streamlit": False
    },
    "license_args": {"openai_key": "YOUR KEY HERE."}
}

In [10]:
framework = uptrain.Framework(cfg_dict=cfg)
ids = framework.log(inputs={"dialogues": golden_dataset['dialogue']}, outputs=None)
ids = framework.log(identifiers=ids, gts=golden_dataset["summary"])

Deleting contents of the folder:  uptrain_smart_data
Deleting contents of the log folder at: uptrain_logs
To start the streamlit dashboard, run the following command:  streamlit run /Users/vipul/Downloads/uptrain_repos/uptrain/uptrain/core/classes/logging/new_st_run.py  -- uptrain_logs
Average cosine_distance is 0.2782163143157959 on your model.
Average l2_distance is 0.5564326643943787 on your model.
Average openai_score is 70.0 on your model.
